# Create loadings raster

We need to convert a shapefile joined to a CSV (`loadings.gpkg`, done in QGis) file of loading data to a) a raster with b) values per area.

Converting to a raster is relatively simple, though a pain to get right the first time.

Getting the area is a bit trickier; we will follow the approach in [this Stack Exchange GIS post](https://gis.stackexchange.com/questions/269518/auto-select-suitable-utm-zone-based-on-grid-intersection) and get the [UTM zone](https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system) for each raster cell, and then use the projected coordinates to calculate the area of each cell.

If you are curious, the [real fun math is here](https://gis.stackexchange.com/questions/127165/more-accurate-way-to-calculate-area-of-rasters).

In [ ]:
from functools import partial
from rasterio.transform import from_origin
from shapely.geometry import mapping, shape
from shapely.ops import transform
import fiona
import math
import numpy as np
import pyproj
import rasterio

In [ ]:
def get_epsg_utm_code(obj):
    lon, lat = obj.centroid.x, obj.centroid.y
    utm_band = str((math.floor((lon + 180) / 6 ) % 60) + 1)
    if len(utm_band) == 1:
        utm_band = '0'+utm_band
    if lat >= 0:
        epsg_code = '326' + utm_band
    else:
        epsg_code = '327' + utm_band
    return epsg_code

In [ ]:
def project_obj(obj):
    # From https://gis.stackexchange.com/questions/127427/transforming-shapely-polygon-and-multipolygon-objects
    project = partial(
        pyproj.transform,
        pyproj.Proj(init='epsg:4326'), 
        pyproj.Proj(init='epsg:{0}'.format(get_epsg_utm_code(obj)))
    )
    return transform(project, obj)

In [ ]:
def to_array_indices(obj):
    lon, lat = obj.centroid.x - 0.25, obj.centroid.y + 0.25
    row = (90 - lat) / 0.5 
    col = (180 + lon) / 0.5
    return int(row), int(col)

First, write raster with the areas to make sure we didn't make any obvious mistakes

In [ ]:
arr = np.zeros((180 * 2,360 * 2)).astype(np.float) - 1

with fiona.open("loadings.gpkg") as f:
    for feat in f:
        obj = shape(feat['geometry'])
        arr[to_array_indices(obj)] = project_obj(obj).area

new_dataset = rasterio.open('areas.tif', 'w', 
                            driver='GTiff',
                            height = arr.shape[0], 
                            width = arr.shape[1],
                            count=1, 
                            dtype=str(arr.dtype),
                            nodata=-1,
                            crs='+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs ',
                            transform=from_origin(-180, 90, 0.5, 0.5))

new_dataset.write(arr, 1)
new_dataset.close()

Next, write the loadings divided by area

In [ ]:
arr = np.zeros((180 * 2,360 * 2)).astype(np.float) - 1

with fiona.open("loadings.gpkg") as f:
    for feat in f:
        obj = shape(feat['geometry'])
        arr[to_array_indices(obj)] = (
            float(feat['properties']['Loading_per_square_Loading']) /
            project_obj(obj).area
        )

new_dataset = rasterio.open('loadings.tif', 'w', 
                            driver='GTiff',
                            height = arr.shape[0], 
                            width = arr.shape[1],
                            count=1, 
                            dtype=str(arr.dtype),
                            nodata=-1,
                            crs='+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs ',
                            transform=from_origin(-180, 90, 0.5, 0.5))

new_dataset.write(arr, 1)
new_dataset.close()